In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go

# accelerometer data

- use calibrated data to set values in beginning to 0
- that way we loose the gravity vector

In [ ]:
acc = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/2024-11-09_15-06-37/Accelerometer.csv')

In [ ]:
fig=plt.figure()
acc['z'].plot()
acc['y'].plot()
acc['x'].plot()
plt.legend()

In [ ]:
grav = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/2024-11-09_15-06-37/Gravity.csv')

In [ ]:
grav['z'].plot()
grav['y'].plot()
grav['x'].plot()

# gyroscope data

- raw data is better than reconstructed angles
- use calibrated data to set values in beginning to 0

In [ ]:
gyro = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/2024-11-09_15-06-37/Gyroscope.csv')

In [ ]:
gyro['time_diff_s'] = gyro['time'].diff().fillna(0) / 1e9
gyro['integral_x'] = (gyro['x']*gyro['time_diff_s']).cumsum()
gyro['integral_y'] = (gyro['y']*gyro['time_diff_s']).cumsum()
gyro['integral_z'] = (gyro['z']*gyro['time_diff_s']).cumsum()

In [ ]:
gyro

In [ ]:
fig = plt.figure()
gyro['x'].plot()
gyro['y'].plot()
gyro['z'].plot()
fig.legend()

fig1 = plt.figure()
gyro['integral_x'].plot()
gyro['integral_y'].plot()
gyro['integral_z'].plot()
fig1.legend()

In [ ]:
ori = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/2024-11-09_15-06-37/Orientation.csv')
ori['yaw'].plot()
ori['roll'].plot()
ori['pitch'].plot()

In [ ]:
ori

In [ ]:
gyro

# double integration

In [ ]:
def integrate_gyro(gyro_data, dt):
    rotation = np.array([0, 0, 0])
    for gyro in gyro_data:
        rotation += gyro * dt
    return rotation

def integrate_accel(accel_data, dt):
    velocity = np.array([0, 0, 0])
    position = np.array([0, 0, 0])
    for accel in accel_data:
        velocity += accel * dt
        position += velocity * dt
    return position


In [ ]:
def doubleIntegrate(df):
    df = df.copy()
    if 'ax' in df.columns:
        df.drop('ax',axis=1,inplace=True)
    if 'az' in df.columns:
        df.drop('az',axis=1,inplace=True)
    if 'ay' in df.columns:
        df.drop('ay',axis=1,inplace=True)
    df.rename(columns={'x': 'ax','y': 'ay','z': 'az'}, inplace=True)
    df['time_diff_s'] = df['time'].diff().fillna(0) / 1e9
    df['vx'] = df['ax'].cumsum()*df['time_diff_s']
    df['x'] = df['vx'].cumsum()*df['time_diff_s']

    df['vy'] = df['ay'].cumsum()*df['time_diff_s']
    df['y'] = df['vy'].cumsum()*df['time_diff_s']

    df['vz'] = df['az'].cumsum()*df['time_diff_s']
    df['z'] = df['vz'].cumsum()*df['time_diff_s']
    return df

In [ ]:
gyro2 = doubleIntegrate(gyro)
gyro2[['vx','vy','vz']].plot()

In [ ]:
grav2 = doubleIntegrate(grav)
grav2[['x','y','z']].plot()

In [ ]:
grav

# reconstruct location in 3D with precalculated yaw,pitch,roll and gravity

In [ ]:
ori = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/2024-11-09_15-06-37/Orientation.csv')
ori.rename(columns={'yaw': 'pitchn','roll': 'yawn','pitch':'rolln'}, inplace=True)
ori.rename(columns={'rolln': 'roll','pitchn':'pitch','yawn':'yaw'}, inplace=True)

fig = plt.figure()
ori['yaw'].plot() # = pitch_new
ori['roll'].plot() #yaw_new
ori['pitch'].plot() #=roll_new
fig.legend()

# around y = pitch
# around x = roll
# around z = yaw

# z = forward
# y = down
# x = left/right

In [ ]:
grav = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/2024-11-09_15-06-37/Gravity.csv')
fig = plt.figure()
grav['z'].plot()
grav['y'].plot()
grav['x'].plot()

fig.legend()


In [ ]:
motion = pd.merge(ori, grav, on=['time','seconds_elapsed'],how='inner')
motion.rename(columns={'x': 'ax','y': 'ay','z': 'az'}, inplace=True)
motion

In [ ]:
def euler_to_rotation_matrix(yaw, pitch, roll):
    # Convert degrees to radians if necessary
    #yaw = np.radians(yaw)
    #pitch = np.radians(pitch)
    #roll = np.radians(roll)

    # Rotation matrices around each axis
    R_yaw = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                      [np.sin(yaw), np.cos(yaw), 0],
                      [0, 0, 1]])

    R_pitch = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                        [0, 1, 0],
                        [-np.sin(pitch), 0, np.cos(pitch)]])

    R_roll = np.array([[1, 0, 0],
                       [0, np.cos(roll), -np.sin(roll)],
                       [0, np.sin(roll), np.cos(roll)]])

    # Combined rotation matrix (R = R_yaw * R_pitch * R_roll)
    R = R_yaw @ R_pitch @ R_roll
    return R

def transform_acceleration_to_global(ax,ay,az, yaw, pitch, roll):
    R = euler_to_rotation_matrix(yaw, pitch, roll)
    #print(R, np.array((ax,ay,az)).reshape(3,1))
    global_accel = R @ np.array((ax,ay,az)).reshape(3,1)
    return global_accel.flatten()

motion[['global_ax', 'global_ay', 'global_az']] = motion.apply(
    lambda row: transform_acceleration_to_global(row['ax'], row['ay'], row['az'],
                                                  row['yaw'], row['pitch'], row['roll']),
    axis=1,
    result_type='expand'  # This allows us to expand the result into multiple columns
)
#compensate gravity
#motion['global_ay'] += 9.80665


In [ ]:
fig2 = plt.figure()
motion['global_ax'].plot()
motion['global_ay'].plot()
motion['global_az'].plot()
fig2.legend()

#fig3 = plt.figure()
#motion['yaw'].plot()
#motion['roll'].plot()
##motion['global_az'].plot()
#fig3.legend()

In [ ]:
motion['time_diff_s'] = motion['time'].diff().fillna(0) / 1e9
motion['vx'] = (motion['global_ax']*motion['time_diff_s']).cumsum()
motion['vy'] = (motion['global_ay']*motion['time_diff_s']).cumsum()
motion['vz'] = (motion['global_az']*motion['time_diff_s']).cumsum()
motion['x'] = (motion['global_ax']*motion['time_diff_s']*motion['time_diff_s']*0.5 + motion['vx']*motion['time_diff_s']).cumsum()
motion['y'] = (motion['global_ay']*motion['time_diff_s']*motion['time_diff_s']*0.5 + motion['vy']*motion['time_diff_s']).cumsum()
motion['z'] = (motion['global_az']*motion['time_diff_s']*motion['time_diff_s']*0.5 + motion['vz']*motion['time_diff_s']).cumsum()

fig2 = plt.figure()
motion['vz'].plot()
motion['vy'].plot()
motion['vx'].plot()
fig2.legend()

fig3 = plt.figure()
motion['z'].plot()
motion['y'].plot()
motion['x'].plot()
fig3.legend()

# reconstruction with calibrated data

In [ ]:
gyro = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/2024-11-09_15-06-37/Gyroscope.csv')
gyro['time_diff_s'] = gyro['time'].diff().fillna(0) / 1e9
gyro['roll'] = (gyro['x']*gyro['time_diff_s']).cumsum()
gyro['pitch'] = (gyro['y']*gyro['time_diff_s']).cumsum()
gyro['yaw'] = (gyro['z']*gyro['time_diff_s']).cumsum()
gyro.drop(['x','y','z'],axis=1,inplace=True)

acc = pd.read_csv('/Users/matthiaskargl/Codes/SLAM/sample_data/2024-11-09_15-06-37/Accelerometer.csv')

motion = pd.merge(gyro, acc, on=['time','seconds_elapsed'],how='inner')
motion.rename(columns={'x': 'ax','y': 'ay','z': 'az'}, inplace=True)
motion

In [ ]:
def euler_to_rotation_matrix(yaw, pitch, roll):
    # Convert degrees to radians if necessary
    #yaw = np.radians(yaw)
    #pitch = np.radians(pitch)
    #roll = np.radians(roll)

    # Rotation matrices around each axis
    R_yaw = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                      [np.sin(yaw), np.cos(yaw), 0],
                      [0, 0, 1]])

    R_pitch = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                        [0, 1, 0],
                        [-np.sin(pitch), 0, np.cos(pitch)]])

    R_roll = np.array([[1, 0, 0],
                       [0, np.cos(roll), -np.sin(roll)],
                       [0, np.sin(roll), np.cos(roll)]])

    # Combined rotation matrix (R = R_yaw * R_pitch * R_roll)
    R = R_yaw @ R_pitch @ R_roll
    return R

def transform_acceleration_to_global(ax,ay,az, yaw, pitch, roll):
    R = euler_to_rotation_matrix(yaw, pitch, roll)
    #print(R, np.array((ax,ay,az)).reshape(3,1))
    global_accel = R @ np.array((ax,ay,az)).reshape(3,1)
    return global_accel.flatten()

motion[['global_ax', 'global_ay', 'global_az']] = motion.apply(
    lambda row: transform_acceleration_to_global(row['ax'], row['ay'], row['az'],
                                                  row['yaw'], 
                                                  row['pitch'], 
                                                  row['roll']
                                                  ),
    axis=1,
    result_type='expand'  # This allows us to expand the result into multiple columns
)
#compensate gravity
#motion['global_ay'] += 9.80665


In [ ]:
motion['time_diff_s'] = motion['time'].diff().fillna(0) / 1e9
motion['vx'] = (motion['global_ax']*motion['time_diff_s']).cumsum()
motion['vy'] = (motion['global_ay']*motion['time_diff_s']).cumsum()
motion['vz'] = (motion['global_az']*motion['time_diff_s']).cumsum()
motion['x'] = (motion['global_ax']*motion['time_diff_s']*motion['time_diff_s']*0.5 + motion['vx']*motion['time_diff_s']).cumsum()
motion['y'] = (motion['global_ay']*motion['time_diff_s']*motion['time_diff_s']*0.5 + motion['vy']*motion['time_diff_s']).cumsum()
motion['z'] = (motion['global_az']*motion['time_diff_s']*motion['time_diff_s']*0.5 + motion['vz']*motion['time_diff_s']).cumsum()

fig0 = plt.figure()
motion['ax'].plot()
motion['ay'].plot()
motion['az'].plot()
fig0.legend()

fig1 = plt.figure()
motion['global_ax'].plot()
motion['global_ay'].plot()
motion['global_az'].plot()
fig1.legend()

fig2 = plt.figure()
motion['vz'].plot()
motion['vy'].plot()
motion['vx'].plot()
fig2.legend()

fig3 = plt.figure()
motion['z'].plot()
motion['y'].plot()
motion['x'].plot()
fig3.legend()

# noise reduction via fourier transform

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def denoiseFFT(signal, fs = 1000, cutoff_frequency = 60, plot=False):
    # Apply FFT
    fft_signal = np.fft.fft(signal)
    frequencies = np.fft.fftfreq(motion.shape[0], 1/fs) 

    # Create a mask for filtering (e.g., remove frequencies above 60 Hz)
    fft_signal[np.abs(frequencies) > cutoff_frequency] = 0  

    # Inverse FFT to get the filtered signal
    filtered_signal = np.fft.ifft(fft_signal)   

    if plot:
        plt.figure(figsize=(12, 6))
        plt.subplot(3, 1, 1)
        plt.title('Original Signal with Noise')
        plt.plot(motion['ax'])
        plt.subplot(3, 1, 2)
        plt.title('Filtered Signal')
        plt.plot(filtered_signal.real)  # Use .real since IFFT may produce complex numbers
        plt.subplot(3, 1, 3)
        plt.title('Frequencies')
        plt.plot(fft_signal)  # Use .real since IFFT may produce complex numbers
        plt.tight_layout()
        plt.show()
    
    return filtered_signal.real

denoiseFFT(motion['global_az'], 1000, 60, True)


# 3d trajectory

In [ ]:
# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the trajectory
ax.plot(motion['x'], motion['y'], motion['z'], label='Trajectory', color='b')
ax.set_title('3D Trajectory Plot')
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
ax.set_zlabel('Z Position')
ax.legend()

# Show the plot
plt.show()

In [ ]:
# Create a scatter plot for the trajectory
fig = go.Figure(data=[go.Scatter3d(
    x=motion['x'],
    y=motion['y'],
    z=motion['z'],
    mode='lines',
    line=dict(color='blue', width=2),
)])

fig.update_layout(
    title='3D Trajectory Plot with Grid',
    scene=dict(
        xaxis=dict(title='X Position', gridcolor='lightgray', showgrid=True),
        yaxis=dict(title='Y Position', gridcolor='lightgray', showgrid=True),
        zaxis=dict(title='Z Position', gridcolor='lightgray', showgrid=True),
        aspectmode='cube'  # Optional: Keep the aspect ratio equal
    ),
    margin=dict(l=0, r=0, b=0, t=40)  # Adjust margins for better visibility
)

# Show the interactive plot
fig.show()